In [ ]:
import QuantLib as ql

In [ ]:
import csv
import os
from typing import List, Tuple
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib.ticker import StrMethodFormatter


def read_swap_quotes(date) -> List[ql.RelinkableQuoteHandle]:
    root_dir = os.getcwd()
    date_as_int = date.year() * 10000 + date.month() * 100 + date.dayOfMonth()
    file_name = 'swap_rates_' + str(date_as_int) + '.csv'
    file_path = os.path.join(root_dir, 'data/', file_name)
    with open(file_path, 'rt') as file:
        reader = csv.reader(file, delimiter=',')
        rates = [(ql.PeriodParser.parse(str(r[0])), float(r[1])) for r in reader]
        return [(q[0], ql.RelinkableQuoteHandle(ql.SimpleQuote(q[1]))) for q in rates]


def read_liabilities() -> List[Tuple[ql.Date, float]]:
    root_dir = os.getcwd()
    file_name = 'liabilities.csv'
    file_path = os.path.join(root_dir, 'data/', file_name)
    with open(file_path, 'rt') as file:
        reader = csv.reader(file, delimiter=',')
        return [(ql.Date(str(r[0]), '%Y%m%d'), float(r[1])) for r in reader]
    

def calculate_last_liquid_forward(crv: ql.YieldTermStructureHandle, fsp: ql.Period):
    dc = crv.dayCounter()
    omega = 8.0 / 15.0
    fsp_date = crv.referenceDate() + fsp
    cut_off = crv.timeFromReference(fsp_date)
    llfr_weights = ((ql.Period(25, ql.Years), 1.0), 
                    (ql.Period(30, ql.Years), 0.5), 
                    (ql.Period(40, ql.Years), 0.25), 
                    (ql.Period(50, ql.Years), 0.125))
    llfr = 0.0
    for tenor, weight in llfr_weights:
        time_to_maturity = crv.timeFromReference(fsp_date + tenor)
        llfr += weight * crv.forwardRate(
            cut_off, time_to_maturity, ql.Continuous, ql.NoFrequency, True).rate()
    return llfr


def bump_quote(quote_handle: ql.RelinkableQuoteHandle, bump = 0.0001):
    quote_handle.linkTo(ql.SimpleQuote(quote_handle.value() + bump))

    
def liabilities_pricer(discount_handle: ql.YieldTermStructureHandle):
    
    def value(cash_flows: List[Tuple[ql.Date, float]]) -> float:
        return sum([discount_handle.discount(dt) * amount for dt, amount in cash_flows])

    return value


def plot_zero_curve(curve_handle: ql.YieldTermStructureHandle, tenors: List[ql.Period], title):
    reference_date = curve_handle.referenceDate()
    d_counter = curve_handle.dayCounter()
    dates = [reference_date + tnr for tnr in tenors]
    t = [curve_handle.timeFromReference(d) for d in dates]
    z = [curve_handle.zeroRate(d, d_counter, ql.Continuous, ql.NoFrequency).rate() for d in dates]
    fig, ax = plt.subplots()
    ax.plot(t, z)
    ax.set(xlabel='time', ylabel='zero', title=title)
    ax.grid()
    plt.show()

In [ ]:
# SWAP INDEX CONVENTIONS

SETTLEMENT_DAYS = 2
BUSINESS_CONVENTION = ql.Unadjusted
DAY_COUNT = ql.SimpleDayCounter()
CALENDAR = ql.NullCalendar()
CCY = ql.EURCurrency()
FXD_FREQUENCY = ql.Annual
FLT_TENOR = ql.Period(6, ql.Months)


# FTK CURVE CONSTRUCTION FUNCTION

def build_ftk_curve(valuation_date: ql.Date, quote_handles: List[ql.QuoteHandle]):
    idx = ql.IborIndex("FTK_IDX", FLT_TENOR, SETTLEMENT_DAYS, CCY, CALENDAR, BUSINESS_CONVENTION, False, DAY_COUNT)
    settlement = CALENDAR.advance(today, SETTLEMENT_DAYS, ql.Days)
    instruments = [ql.SwapRateHelper(q, t, CALENDAR, FXD_FREQUENCY, BUSINESS_CONVENTION, DAY_COUNT, idx) 
                   for t, q in quote_handles]
    crv = ql.PiecewiseLogLinearDiscount(settlement, instruments, DAY_COUNT)
    crv.enableExtrapolation()
    return crv


In [ ]:
# UFR CONVENTIONS

FIRST_SMOOTHING_POINT = ql.Period(20, ql.Years)
ALPHA = 0.1;

ufr_compounded = ql.InterestRate(0.023, DAY_COUNT, ql.Compounded, ql.Annual)
ufr_continuous = ufr_compounded.equivalentRate(ql.Continuous, ql.Annual, 1.0).rate()

In [ ]:
# VALUATION DATE

today = CALENDAR.adjust(ql.Date(29, ql.March, 2019))
ql.Settings.instance().evaluationDate = today


# READ SWAP RATES

swap_quotes = read_swap_quotes(today)

# UFR CURVE 2015 CONSTRUCTION
    
ftk_handle = ql.RelinkableYieldTermStructureHandle()
llfr_handle = ql.RelinkableQuoteHandle()
ufr_handle = ql.QuoteHandle(ql.SimpleQuote(ufr_continuous))

def update_llfr_handle():
    llfr = calculate_last_liquid_forward(ftk_handle, FIRST_SMOOTHING_POINT)
    llfr_handle.linkTo(ql.SimpleQuote(llfr))
    print('LLFR observer notified.')
    

llfr_observer = ql.Observer(update_llfr_handle)
for _, quote_handle in swap_quotes:
    llfr_observer.registerWith(quote_handle)

ftk_handle.linkTo(build_ftk_curve(today, swap_quotes))
update_llfr_handle()
ufr_crv = ql.UltimateForwardTermStructure(ftk_handle, llfr_handle, ufr_handle, FIRST_SMOOTHING_POINT, ALPHA)

ufr_handle = ql.YieldTermStructureHandle(ufr_crv)

In [ ]:
# print(swap_quotes[15][1].value())
# bump_quote(swap_quotes[15][1], 0.001)
# print(swap_quotes[15][1].value())

ftk_handle.discount(ql.Date(31, ql.December, 2020))

In [ ]:
liabilities = read_liabilities()

ftk_pricer = liabilities_pricer(ftk_handle)
ufr_pricer = liabilities_pricer(ufr_handle)


In [ ]:
npv_ftk = ftk_pricer(liabilities)
print('FTK NPV ', npv_ftk)

npv_ufr = ufr_pricer(liabilities)
print('UFR NPV ', npv_ufr)

In [ ]:
tenors = [ql.PeriodParser.parse(str(i) + 'y') for i in range(1, 101)]

In [ ]:
plot_zero_curve(ufr_handle, tenors, 'UFR 2015 zeroes')